In [1]:
import numpy as np
class l1_regularization():
    #Lasso Regression
    def __init__(self,alpha):
        self.alpha=alpha
    def __call__(self,w):
        return self.alpha*np.linalg.norm(w)
    def grad(self,w):
        return self.alpha*np.sign(w)
    
        

In [2]:
class l2_regularization():
    #Ridge Regression
    def __init__(self,alpha):
        self.alpha=alpha
    def __call__(self,w):
        return 0.5*self.alpha*(w.T.dot(w))
    def grad(self,w):
        return self.alpha*w
    

In [3]:
def l1_l2_regularization():
    #Elatic Net
    def __init__(self,alpha,r=0.5):
        self.r=r
        self.alpha=alpha
    def __call__(self,w):
        out_1=r*self.alpha*np.linalg.norm(w)
        out_2=(1-r)*0.5*self.alpha*(w.T.dot(w))
        out=out_1+out_2
        return out
    def grad(self,w):
        out_1=r*self.alpha*np.sign(w)
        out_2=(1-r)*self.alpha*w
        return (out_1+out_2)
    
        

In [4]:
import math

In [5]:
class Regression(object):
    """ Base regression model. Models the relationship between a scalar dependent variable y and the independent 
    variables X. 
    Parameters:
    -----------
    n_iterations: float
        The number of training iterations the algorithm will tune the weights for.
    learning_rate: float
        The step length that will be used when updating the weights.
    """
    def __init__(self, n_iterations, learning_rate):
        self.n_iterations = n_iterations
        self.learning_rate = learning_rate

    def initialize_weights(self, n_features):
        """ Initialize weights randomly [-1/N, 1/N] """
        limit = 1 / math.sqrt(n_features)
        self.w = np.random.uniform(-limit, limit, (n_features, ))

    def fit(self, X, y):
        # Insert constant ones for bias weights
        X = np.insert(X, 0, 1, axis=1)
        self.training_errors = []
        self.initialize_weights(n_features=X.shape[1])

        # Do gradient descent for n_iterations
        for i in range(self.n_iterations):
            y_pred = X.dot(self.w)
            # Calculate l2 loss
            mse = np.mean(0.5 * (y - y_pred)**2 + self.regularization(self.w))
            self.training_errors.append(mse)
            # Gradient of l2 loss w.r.t w
            grad_w = -(y - y_pred).dot(X) + self.regularization.grad(self.w)
            # Update the weights
            self.w -= self.learning_rate * grad_w

    def predict(self, X):
        # Insert constant ones for bias weights
        X = np.insert(X, 0, 1, axis=1)
        y_pred = X.dot(self.w)
        return y_pred

class LinearRegression(Regression):
    """Linear model.
    Parameters:
    -----------
    n_iterations: float
        The number of training iterations the algorithm will tune the weights for.
    learning_rate: float
        The step length that will be used when updating the weights.
    gradient_descent: boolean
        True or false depending if gradient descent should be used when training. If 
        false then we use batch optimization by least squares.
    """
    def __init__(self, n_iterations=100, learning_rate=0.001, gradient_descent=True):
        self.gradient_descent = gradient_descent
        # No regularization
        self.regularization = lambda x: 0
        self.regularization.grad = lambda x: 0
        super(LinearRegression, self).__init__(n_iterations=n_iterations,
                                            learning_rate=learning_rate)
    def fit(self, X, y):
        # If not gradient descent => Least squares approximation of w
        if not self.gradient_descent:
            # Insert constant ones for bias weights
            X = np.insert(X, 0, 1, axis=1)
            # Calculate weights by least squares (using Moore-Penrose pseudoinverse)
            U, S, V = np.linalg.svd(X.T.dot(X))
            S = np.diag(S)
            X_sq_reg_inv = V.dot(np.linalg.pinv(S)).dot(U.T)
            self.w = X_sq_reg_inv.dot(X.T).dot(y)
        else:
            super(LinearRegression, self).fit(X, y)

In [6]:
class LassoRegression(Regression):
    """Linear regression model with a regularization factor which does both variable selection 
    and regularization. Model that tries to balance the fit of the model with respect to the training 
    data and the complexity of the model. A large regularization factor with decreases the variance of 
    the model and do para.
    Parameters:
    -----------
    degree: int
        The degree of the polynomial that the independent variable X will be transformed to.
    reg_factor: float
        The factor that will determine the amount of regularization and feature
        shrinkage. 
    n_iterations: float
        The number of training iterations the algorithm will tune the weights for.
    learning_rate: float
        The step length that will be used when updating the weights.
    """
    def __init__(self, reg_factor, n_iterations=3000, learning_rate=0.01):
        self.regularization = l1_regularization(alpha=reg_factor)
        super(LassoRegression, self).__init__(n_iterations, 
                                            learning_rate)

    def fit(self, X, y):
        #X = normalize(polynomial_features(X, degree=self.degree))
        super(LassoRegression, self).fit(X, y)

    def predict(self, X):
        #X = normalize(polynomial_features(X, degree=self.degree))
        return super(LassoRegression, self).predict(X)

In [7]:
class Ridge_regression(Regression):
    def __init__(self,reg_factor,n_iterations,learning_rate):
        self.regularization=l2_regularization(alpha=reg_factor)
        super(Ridge_regression,self).__init__(n_iterations,learning_rate)

In [14]:
def main():
    from sklearn.datasets import make_regression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import r2_score
    X, y = make_regression(n_samples=100, n_features=1, noise=20)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

    model =Ridge_regression(reg_factor=0.05,n_iterations=4000,learning_rate=0.001)

    model.fit(X_train, y_train)
    pred_train=model.predict(X_train)
    pred_test=model.predict(X_test)
    print(r2_score(y_train,pred_train))
    print(r2_score(y_test,pred_test))

In [15]:
main()

0.8421630466393585
0.8097339346421636
